In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta

In [2]:
faults = pd.read_csv("../data/J1939Faults.csv", low_memory = False)
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")

C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
faults_prepped = faults.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [5]:
exclude_coords = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.7  # Radius of Earth in km - roughly 3958.7 miles
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2 # this is the haversine formula that's accounting for the curvature of the earth. think calculus & a polar coordinate system.
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)) # we're calculating the central angle between the two points
    return R * c # calculates the arc length (distance) we're multiplying the radius of earth and the central angle
threshold = 0.5  # Distance in km - this is roughly about 3.1 miles
distances = np.array([
    haversine(faults_prepped['Latitude'], faults_prepped['Longitude'], lat, lon)
    for lat, lon in exclude_coords
]) # I'm calculating the difference between the excluded coordinates and geo coordinate in my dataframe. the array is contains the distances as scalar quantities (floats).
# which rows are outside all vicinities
outside_vicinity = (distances > threshold).all(axis=0) # boolean array true if its outside vicinity
faults_prepped = faults_prepped[outside_vicinity]

In [6]:
diagnostics_pivot = diagnostics.pivot(index=['FaultId'], columns='Name', values='Value')

In [7]:
data = diagnostics_pivot.merge(faults_prepped, left_on = 'FaultId', right_on = 'RecordID', how = 'inner')

In [8]:
data.sample(5)

,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
269174,0,14.645,True,64.6226,45668.78,179.6,52,40.02,210.425,1321.375,891.75,100,6236.837975668,9.325302,32,True,91.4,1023,NaN,NaN,65.07893,NaN,100,9.57,300478,6133840,2015-12-04 10:55:13.000,Low (Severity Low) Engine Coolant Level,05317106*05005224*050815201656*09400035*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,111,17,True,126,1814,105369518,36.106851,-86.410185,2015-12-04 10:55:50.000
754135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,NaN,859137,24846952,2017-08-28 10:47:33.000,Abnormal Update Rate Tire Location,NaN,NaN,CECU3B-NAMUX4,PACCR,49,929,9,False,126,1646,105420184,33.633842,-84.499861,2017-08-28 10:47:29.000
1056621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,1247985,123658795,2020-03-05 07:36:01.000,Low (Severity Medium) Engine Coolant Level,04384413*22038252*120718181232*60701742*G1*BGT*,80007289,6X1u17D1500000000,CMMNS,0,111,18,False,13,2084,108086753,38.423888,-78.879814,2020-03-05 07:35:57.000
567376,27.6,14.645,False,64.6226,175313.7,186.8,26,35.38,209.75,883.75,3406.95,44.4,23772.050443324,1.637872,32,True,102.2,1023,NaN,NaN,31.96178,NaN,100,2.61,646341,13011638,2016-11-28 20:59:09.000,Low (Severity Low) Engine Coolant Level,05317106*05005224*050815201656*09400035*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,111,17,True,126,1814,105369518,30.338379,-81.705509,2016-11-28 20:59:46.000
405637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,455965,8762232,2016-04-21 18:03:43.000,Low (Severity Low) Engine Coolant Level,05317106*05030782*050815201656*09400035*G1*BDR*,79857688,6X1u13D1500000000,CMMNS,0,111,17,False,126,1820,105465571,35.958425,-86.445231,2016-04-21 18:03:39.000


In [9]:
columns_to_change = ['AcceleratorPedal', 
                     'BarometricPressure',
                     'CruiseControlSetSpeed',
                     'DistanceLtd',
                     'EngineCoolantTemperature',
                     'EngineLoad',
                     'EngineOilPressure',
                     'EngineOilTemperature',
                     'EngineRpm',
                     'EngineTimeLtd',
                     'FuelLevel',
                     'FuelLtd',
                     'FuelRate',
                     'FuelTemperature',
                     'IntakeManifoldTemperature',
                     'Speed',
                     'SwitchedBatteryVoltage',
                     'TurboBoostPressure',
                    ]   

In [10]:
for column in columns_to_change:
    data[column] = data[column].str.replace(',', '.').astype(np.float64)

In [11]:
data['EngineRpm'].describe()

count    531324.000000
mean        977.688730
std         404.159394
min           0.000000
25%         601.375000
50%        1078.625000
75%        1342.250000
max        2715.625000
Name: EngineRpm, dtype: float64

In [12]:
data.loc[data['spn'] == 1569]['EngineRpm'].describe()

count    4959.000000
mean     1076.714862
std       339.198449
min         0.000000
25%       650.750000
50%      1221.875000
75%      1357.312500
max      1881.625000
Name: EngineRpm, dtype: float64

In [13]:
data.loc[data['spn'] == 5246]['EngineRpm'].describe()

count     463.000000
mean      701.558045
std       424.685256
min         0.000000
25%       598.062500
50%       648.750000
75%      1002.500000
max      1626.500000
Name: EngineRpm, dtype: float64

In [14]:
data.loc[data['spn'] != 5246 & 1569]['EngineRpm'].describe()

count    530999.000000
mean        978.035437
std         403.960955
min           0.000000
25%         601.375000
50%        1079.250000
75%        1342.375000
max        2715.625000
Name: EngineRpm, dtype: float64

In [15]:
data['LocationTimeStamp'] = pd.to_datetime(data['LocationTimeStamp'])
data['EventTimeStamp'] = pd.to_datetime(data['EventTimeStamp'])

In [16]:
data = data.drop(['ecuModel','ecuMake', 'MCTNumber', 'ServiceDistance'], axis=1)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057049 entries, 0 to 1057048
Data columns (total 38 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   AcceleratorPedal           480660 non-null   float64       
 1   BarometricPressure         530902 non-null   float64       
 2   CruiseControlActive        520765 non-null   object        
 3   CruiseControlSetSpeed      521823 non-null   float64       
 4   DistanceLtd                530760 non-null   float64       
 5   EngineCoolantTemperature   530890 non-null   float64       
 6   EngineLoad                 530421 non-null   float64       
 7   EngineOilPressure          531008 non-null   float64       
 8   EngineOilTemperature       529370 non-null   float64       
 9   EngineRpm                  531324 non-null   float64       
 10  EngineTimeLtd              527047 non-null   float64       
 11  FuelLevel                  455471 non

In [36]:
data['target_time_min'] = data.loc[data['spn'] == 5246]['EventTimeStamp'] - pd.Timedelta(hours=2) 
data['target_time_max'] = data.loc[data['spn'] == 5246]['EventTimeStamp'] - pd.Timedelta(hours=3)

start_times = data['target_time_max'].dropna()
end_times = data['target_time_min'].dropna()

In [72]:
def get_datetimes_between_indices(series1, series2, dataframe, datetime_column):
    datetimes = []
    for i in range(min(len(series1), len(series2)) - 1):
        start_index = min(series1.index[i], series2.index[i])
        end_index = max(series1.index[i + 1], series2.index[i + 1])

        values_between = dataframe.loc[start_index:end_index, datetime_column]
        datetimes.extend(values_between)
    return datetimes

result = get_datetimes_between_indices(start_times, end_times, data, 'EventTimeStamp')

In [80]:
data['target'] = data['EventTimeStamp'].isin(result)

In [84]:
data.loc[data['target'] == True]

,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,target_time_min,target_time_max,event,target
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4645.45,NaN,33470.466902,NaN,NaN,False,NaN,22527,NaN,NaN,NaN,NaN,NaN,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,0,5246,0,True,1,1630,40.733009,-74.087777,2015-02-23 05:08:23,2015-02-23 03:05:44,2015-02-23 01:05:44,False,True
1801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255,NaN,NaN,NaN,NaN,NaN,2091,1011010,2015-02-23 05:08:27,Abnormal Update Rate Tire Location,unknown,unknown,49,929,9,False,126,1630,40.733009,-74.087777,2015-02-23 05:08:23,NaT,NaT,False,True
1802,100.0,14.645,False,64.62260,470586.3,188.6,100.0,35.96,203.9563,1338.625,9485.05,27.2,70380.718094,16.576850,NaN,True,95.0,1023,NaN,45.76787,3276.75,NaN,27.84,2093,1011019,2015-02-23 05:08:45,Low (Severity Low) Engine Coolant Level,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,0,111,17,True,1,1417,30.725694,-95.575462,2015-02-23 05:09:22,NaT,NaT,False,True
1803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18431,NaN,NaN,NaN,NaN,NaN,2094,1011023,2015-02-23 05:09:32,Error in System Engine Exhaust Gas Recirculati...,unknown,unknown,0,3821,11,False,126,1749,37.928981,-79.228564,2015-02-23 05:09:28,NaT,NaT,False,True
1804,0.0,14.790,False,66.48672,239319.8,179.6,13.0,24.94,202.7750,600.375,4645.50,NaN,33470.466902,0.620806,32.0,True,114.8,1279,True,0.00000,3276.75,0,0.29,2095,1011034,2015-02-23 05:09:44,Abnormal Update Rate Tire Location,unknown,unknown,49,929,9,True,126,1630,40.733055,-74.087777,2015-02-23 05:10:21,NaT,NaT,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,1242796,121404042,2020-02-17 17:08:37,NaN,PC4_C1408P151H*,Y043718,0,4346,19,False,1,302,38.192175,-85.707546,2020-02-17 17:08:33,NaT,NaT,False,True
1053101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,1242797,121404043,2020-02-17 17:08:39,Error in System Engine Exhaust Gas Recirculati...,PC4_C1408P151H*,Y043718,0,3821,11,False,1,302,38.192175,-85.707546,2020-02-17 17:08:34,NaT,NaT,False,True
1053102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65535,NaN,NaN,NaN,NaN,NaN,1242798,121404044,2020-02-17 17:08:39,NaN,NaN,NaN,49,5246,19,False,88,302,38.192175,-85.707546,2020-02-17 17:08:34,2020-02-17 15:08:39,2020-02-17 13:08:39,False,True
1053103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,1242799,121404045,2020-02-17 17:08:37,NaN,PC4_C1408P151H*,Y043718,0,5113,9,False,1,302,38.192175,-85.707546,2020-02-17 17:08:33,NaT,NaT,False,True


In [22]:
data.loc[data['spn'] == 5246]

,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,target_time_min,target_time_max,event,target
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4645.45,NaN,33470.466902,NaN,NaN,False,NaN,22527,NaN,NaN,NaN,NaN,NaN,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,0,5246,0,True,1,1630,40.733009,-74.087777,2015-02-23 05:08:23,2015-02-23 03:05:44,2015-02-23 01:05:44,False,False
2589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,2972,1026305,2015-02-23 15:54:22,NaN,unknown,unknown,0,5246,0,True,1,1487,28.077361,-81.897083,2015-02-23 15:54:58,2015-02-23 13:54:22,2015-02-23 11:54:22,False,False
5141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,5714,1070646,2015-02-25 13:53:08,NaN,unknown,unknown,0,5246,0,True,1,1329,39.399583,-82.974768,2015-02-25 13:56:31,2015-02-25 11:53:08,2015-02-25 09:53:08,False,False
5236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,5810,1071907,2015-02-25 14:47:00,NaN,unknown,unknown,0,5246,0,False,1,1329,39.399629,-82.974814,2015-02-25 14:46:56,2015-02-25 12:47:00,2015-02-25 10:47:00,False,False
5949,NaN,14.5000,False,64.62260,441699.6,185.0,10.0,20.30,198.16250,648.125,9087.95,57.2,69605.769379,0.634015,NaN,True,140.0,22527,NaN,0.0,3276.75,NaN,0.58,6535,1097942,2015-02-26 22:24:29,NaN,04993120*00021657*082113134117*07700053*I0*BBZ*,79466573,0,5246,0,True,1,1419,37.596805,-85.865555,2015-02-26 22:25:05,2015-02-26 20:24:29,2015-02-26 18:24:29,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,1239564,119961467,2020-02-06 08:03:09,NaN,04358814*06005963*051718174436*09401683*G1*BDR*,79897320,0,5246,0,False,1,1854,35.943472,-83.823240,2020-02-06 08:03:05,2020-02-06 06:03:09,2020-02-06 04:03:09,False,False
1052278,0.0,13.9925,False,57.78752,477486.8,197.6,10.0,21.46,204.96880,599.500,9344.30,72.4,63966.356499,0.462303,NaN,True,120.2,22527,True,0.0,NaN,100,0.58,1241841,120905759,2020-02-13 13:32:39,NaN,04358814*06026985*051718174436*09401683*G1*BDR*,79903054,0,5246,0,True,1,1872,35.707268,-81.397037,2020-02-13 13:33:15,2020-02-13 11:32:39,2020-02-13 09:32:39,False,False
1052292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,1241858,120910417,2020-02-13 14:01:40,NaN,04358814*06026985*051718174436*09401683*G1*BDR*,79903054,0,5246,0,False,1,1872,35.708101,-81.395648,2020-02-13 13:59:51,2020-02-13 12:01:40,2020-02-13 10:01:40,False,False
1052528,0.0,14.7900,False,0.00000,334736.2,17.6,51.0,84.68,18.10625,651.500,11554.20,100.0,393.220099,2.892693,19.4,True,26.6,255,True,0.0,NaN,0,0.58,1242137,121038018,2020-02-14 11:21:54,NaN,NaN,NaN,49,5246,19,True,88,302,38.349490,-85.708425,2020-02-14 11:22:30,2020-02-14 09:21:54,2020-02-14 07:21:54,False,False


In [23]:
data_csv = pd.to_csv(

SyntaxError: incomplete input (3620138382.py, line 1)